## Types of Query Construction

- **Vector Databases**  
  Queries are primarily based on vector similarity, often using embeddings and nearest neighbor search.

- **Graph Databases**  
  Queries are constructed to traverse nodes and edges, focusing on relationships and connections between entities.

- **Relational Databases**  
  Queries are built using SQL or similar query languages to retrieve structured data based on tables, rows, and columns.


In [ ]:
### first get path where sqlite database will store
# for now i am using docker for sqlite using below command and make one db
# docker run -it --name python_sqlite -v /c/Users/aniln/Desktop/github_celery_redis/Advance_RAG2:/app/data python:3.11-slim bash


In [1]:
import sqlite3
from langchain_community.utilities import SQLDatabase

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('example.db')

# Create a cursor object
cur = conn.cursor()

# Create a new table
cur.execute('''
CREATE TABLE IF NOT EXISTS employees (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    department TEXT
)
''')

# Commit the changes
conn.commit()

# Insert data into the table
cur.execute("INSERT INTO employees (name, age, department) VALUES ('Alice', 30, 'HR')")
cur.execute("INSERT INTO employees (name, age, department) VALUES ('Bob', 24, 'Engineering')")
cur.execute("INSERT INTO employees (name, age, department) VALUES ('Charlie', 28, 'Marketing')")

# Commit the changes
conn.commit()

db = SQLDatabase.from_uri("sqlite:///example.db")

# Print the database dialect
print(db.dialect)

# Print usable table names
print(db.get_usable_table_names())

sqlite
['employees']


In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_community.utilities import SQLDatabase
import os
import getpass

# Connect to your SQLite database
db = SQLDatabase.from_uri("sqlite:///example.db")

os.environ["OPENAI_API_KEY"] = ""  # Replace with your key
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")


# Initialize OpenAI chat model
llm = ChatOpenAI(
    model_name="gpt-4",  # or "gpt-3.5-turbo"
    temperature=0,
)

# Define prompt for SQL generation
system_message = "You are a helpful assistant. Your job is to convert natural language questions to SQL queries. Only give the SQL query."
human_message = "{question}"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", human_message)
])

# Create chain
chain = prompt | llm

# Example query
response = chain.invoke({"question": "How many employees are there?"})
sql_query = response.content
print("Generated SQL:", sql_query)

# Run the SQL on database
result = db.run(sql_query)
print("Result:", result)


Generated SQL: SELECT COUNT(*) FROM employees;
Result: [(6,)]


## Vecotr DB

In [6]:
# Install required packages
# pip install langchain chromadb unstructured openai

from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import os
import getpass

# 1️⃣ Load documents from folder
directory = "C:/Users/aniln/Desktop/github_celery_redis/Advance_RAG2/documents/"  # Replace with your folder path
loader = DirectoryLoader(directory, show_progress=True)
documents = loader.load()

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")


# 2️⃣ Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# 3️⃣ Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 4️⃣ Create vector database and persist locally
persist_directory = "local_vector_db"
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=persist_directory
)


100%|██████████| 1/1 [00:00<00:00, 35.87it/s]


In [7]:

# 5️⃣ Create retriever
vectorstore_retriever = vectordb.as_retriever(search_kwargs={"k": 2})

# 6️⃣ Initialize OpenAI LLM
llm = ChatOpenAI(
    model_name="gpt-4",  # or "gpt-3.5-turbo"
    temperature=0,
)

# 7️⃣ Combine retriever and LLM in a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_retriever,
    return_source_documents=False  # <-- only one output key
)

# 8️⃣ Prompt user query
warning = "Please refrain from speculating if unsure. Simply state 'I don't know'. Keep answers concise (~200 words)."
user_query = input("Enter your query: ")
query = warning + " Requirement: " + user_query

# 9️⃣ Get answer
llm_response = qa_chain.run(query)
print("Response:", llm_response)


Response: I'm sorry, but your question about cricket is not clear. Could you please provide more details or context?
